In [1]:
import os
import sys
sys.path.append(os.pardir) 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Dense, Input, Bidirectional, LSTM, Dropout, CuDNNLSTM
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

import pickle
import codecs
import csv
import struct

from numpy import genfromtxt

Using TensorFlow backend.


In [2]:
# Data file path
Guryongpo_Swell_Data_Path = './Data/1.Guryongpo_Swell(14~17).csv'
Wolopo_Swell_Data_Path = './Data/2.Wolpo_Swell(15~17).csv'
Pohang_Weather_Data_Path = './Data/3.Pohang_Weather(14~17).csv'

In [3]:
# Data list
Guryongpo_Swell = []
Wolopo_Swell = []
Guryongpo_Swell_with_Weather = []
Wolopo_Swell_with_Weather = []

In [4]:
# string list 

# weatehr data column string
c =  ['평균 수온(°C)', '최고 수온(°C)', 
      '최저 수온(°C)', '평균 유의 파고(m)','평균 파고(m)',
      '최고 유의 파고(m)', '최고 최대 파고(m)', '평균 파주기(sec)', 
      '최고 파주기(sec)','평균기온(°C)', '최저기온(°C)','최저기온 시각(hhmi)','최고기온(°C)',
      '최고기온 시각(hhmi)','강수 계속시간(hr)','일강수량(mm)','최대 순간 풍속(m/s)',
      '최대 순간 풍속 풍향(16방위)','최대 순간풍속 시각(hhmi)','최대 풍속(m/s)',
      '최대 풍속 풍향(16방위)','최대 풍속 시각(hhmi)','평균 풍속(m/s)','풍정합(100m)',
      '평균 이슬점온도(°C)','최소 상대습도(%)','최소 상대습도 시각(hhmi)',
      '평균 상대습도(%)','평균 증기압(hPa)','평균 현지기압(hPa)','최고 해면기압(hPa)',
      '최고 해면기압 시각(hhmi)','최저 해면기압(hPa)','최저 해면기압 시각(hhmi)',
      '평균 해면기압(hPa)','가조시간(hr)','합계 일조 시간(hr)','1시간 최다일사 시각(hhmi)',
      '1시간 최다일사량(MJ/m2)','합계 일사(MJ/m2)','평균 전운량(1/10)','평균 중하층운량(1/10)',
      '평균 지면온도(°C)','최저 초상온도(°C)','평균 5cm 지중온도(°C)','평균 10cm 지중온도(°C)',
      '평균 20cm 지중온도(°C)','평균 30cm 지중온도(°C)','0.5m 지중온도(°C)','1.0m 지중온도(°C)',
      '1.5m 지중온도(°C)','3.0m 지중온도(°C)','5.0m 지중온도(°C)','합계 대형증발량(mm)',
      '합계 소형증발량(mm)']
    
# column string data for merging column by column
m = ['G_평균 수온(°C)','W_평균 수온(°C)', 'G_최고 수온(°C)','W_최고 수온(°C)', 
     'G_최저 수온(°C)', 'W_최저 수온(°C)', 'G_평균 유의 파고(m)','W_평균 유의 파고(m)','G_평균 파고(m)',
     'W_평균 파고(m)', 'G_최고 유의 파고(m)','W_최고 유의 파고(m)', 'G_최고 최대 파고(m)','W_최고 최대 파고(m)',
     'G_평균 파주기(sec)','W_평균 파주기(sec)', 'G_최고 파주기(sec)', 'W_최고 파주기(sec)']

In [5]:
# Data after merge
Column_Sum_Swell = []
Row_Sum_Swell = []

In [6]:
# Weather data list
Pohang_Weather = []

In [7]:
# Dataframe 
Guryongpo_DF = []
Wolopo_DF = []
Column_Sum_DF = []
Row_Sum_DF = []
Pohang_DF = []

In [8]:
# load csv file 
def loadData():   
    print("Data loading...")
    
    global Guryongpo_Swell
    global Wolopo_Swell
    global Pohang_Weather
    
    global Guryongpo_Swell_with_Weather
    global Wolopo_Swell_with_Weather
    
    global Guryongpo_DF
    global Wolopo_DF
    global Pohang_DF
    
    # Read csv file from directory
    Guryongpo = genfromtxt(Guryongpo_Swell_Data_Path, delimiter=',' ,encoding='UTF8')
    Wolopo = genfromtxt(Wolopo_Swell_Data_Path, delimiter=',', encoding='UTF8')
    Pohang = genfromtxt(Pohang_Weather_Data_Path, delimiter=',', encoding='UTF8')
    
    
    # Remove colums and row
    Guryongpo_Swell = Guryongpo[1:,3:]
    Wolopo_Swell = Wolopo[1:,3:]
    Pohang_Weather = Pohang[1:,2:]
    
    
    # Convert string to float64
    Guryongpo_Swell.astype('float64')
    Wolopo_Swell.astype('float64')
    Pohang_Weather.astype('float64')
 
   
    # Fill NaN data to 0   
    Guryongpo_Swell[np.isnan(Guryongpo_Swell)] = 0
    Wolopo_Swell[np.isnan(Wolopo_Swell)] = 0
    Pohang_Weather[np.isnan(Pohang_Weather)] = 0

    
    # Guryongpo  - Wolopo
    dis = len(Guryongpo_Swell)- len(Wolopo_Swell)
    
    
    # Concatenate between swell and weather data
    Guryongpo_Swell_with_Weather = np.concatenate([Guryongpo_Swell,Pohang_Weather], axis=1)
    Wolopo_Swell_with_Weather = np.concatenate([Wolopo_Swell,Pohang_Weather[dis:]], axis=1)
    
     
    # Dataframe
    Guryongpo_DF = pd.DataFrame(Guryongpo_Swell_with_Weather, columns = c)
    Wolopo_DF = pd.DataFrame(Wolopo_Swell_with_Weather, columns = c)

   

In [9]:
# 2015-09-25 부터 colum sum 
# 구룡포의 2014 데이터는 어떻게 할 것인지 보류 (2014 구룡포 데이터는 있지만 월포의 데이터가 없음)
# Data merge by column
def sumColumnData():
    print("process colum sum...")
    global Guryongpo_Swell
    global Wolopo_Swell
    global Column_Sum_Swell
    global Column_Sum_DF
    
    global c
    global m
    
    for i in range(len(Guryongpo_Swell)):
        dis = len(Guryongpo_Swell)- len(Wolopo_Swell)
        if(i< dis):
            continue
        else:
            data = []
            for colum in range(len(Guryongpo_Swell[0])):
                g = Guryongpo_Swell[i][colum]
                w = Wolopo_Swell[i-dis][colum]
                data.append(g)
                data.append(w)
            
            for w in range(len(Guryongpo_Swell[0]),len(Guryongpo_Swell_with_Weather[0])):    
                weather = Guryongpo_Swell_with_Weather[i][w]
                data.append(weather)
            Column_Sum_Swell.append(data)    
       
    t = np.concatenate([m,c[9:]])
    Column_Sum_Swell = np.array(Column_Sum_Swell)
    Column_Sum_DF = pd.DataFrame(Column_Sum_Swell, columns = t)

In [10]:
# Data merge by row
def sumRowData():
    global Guryongpo_Swell
    global Wolopo_Swell
    global Row_Sum_Swell
    global Row_Sum_DF
    global c
    
    for i in range(len(Guryongpo_Swell_with_Weather)):
        dis = len(Guryongpo_Swell_with_Weather)- len(Wolopo_Swell_with_Weather)
        if(i< dis):
            g = Guryongpo_Swell_with_Weather[i]
            Row_Sum_Swell.append(g)
        else:
            g = Guryongpo_Swell_with_Weather[i]
            w = Wolopo_Swell_with_Weather[i-dis]
            Row_Sum_Swell.append(g)
            Row_Sum_Swell.append(w)  
            
    Row_Sum_Swell = np.array(Row_Sum_Swell)
    Row_Sum_DF = pd.DataFrame(Row_Sum_Swell, columns = c)
    #

In [11]:
loadData()

Data loading...


In [12]:
Guryongpo_DF[:5]

,평균 수온(°C),최고 수온(°C),최저 수온(°C),평균 유의 파고(m),평균 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec),평균기온(°C),...,평균 10cm 지중온도(°C),평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm)
0,13.7,13.8,13.6,0.6,0.4,0.9,1.2,5.9,6.7,6.9,...,3.8,3.9,4.2,6.5,10.8,13.0,16.7,17.4,0.0,3.3
1,14.0,14.3,13.6,0.7,0.5,0.9,1.3,6.7,8.2,6.1,...,3.9,4.3,4.6,6.6,10.7,12.9,16.6,17.4,0.0,3.1
2,14.4,14.9,14.1,0.6,0.4,1.0,1.4,6.8,8.1,5.2,...,3.6,4.2,4.6,6.7,10.6,12.8,16.5,17.3,0.0,2.2
3,14.5,14.8,13.9,1.3,0.8,2.1,3.3,5.8,6.9,4.3,...,3.8,4.2,4.6,6.7,10.5,12.7,16.4,17.3,0.0,1.5
4,14.4,14.6,14.3,1.5,0.9,2.1,3.0,6.5,6.9,4.6,...,5.1,5.2,5.3,6.8,10.5,12.6,16.4,17.3,0.0,2.0


In [13]:
Wolopo_DF[:5]

,평균 수온(°C),최고 수온(°C),최저 수온(°C),평균 유의 파고(m),평균 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec),평균기온(°C),...,평균 10cm 지중온도(°C),평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm)
0,0.0,24.0,22.0,0.0,0.0,2.5,2.6,0.0,7.3,21.6,...,21.9,21.9,21.9,22.0,21.6,21.3,19.3,16.7,1.8,3.1
1,23.2,23.5,22.9,0.5,0.3,0.8,1.0,4.9,5.4,21.6,...,22.3,22.2,22.1,22.0,21.6,21.3,19.4,16.8,1.8,3.2
2,23.6,24.3,23.2,0.3,0.2,0.5,0.7,4.3,4.8,21.1,...,23.0,22.7,22.5,22.1,21.6,21.2,19.3,16.8,3.3,4.3
3,23.7,24.3,23.3,0.3,0.2,0.4,0.7,4.2,5.0,21.7,...,23.1,22.8,22.7,22.2,21.6,21.2,19.4,16.8,2.7,5.0
4,23.5,23.7,23.4,1.1,0.7,1.8,2.7,4.9,5.8,20.9,...,23.3,23.1,23.0,22.2,21.5,21.2,19.3,16.8,5.1,7.2


In [14]:
sumColumnData()

process colum sum...


In [18]:
Column_Sum_DF[:5]

,G_평균 수온(°C),W_평균 수온(°C),G_최고 수온(°C),W_최고 수온(°C),G_최저 수온(°C),W_최저 수온(°C),G_평균 유의 파고(m),W_평균 유의 파고(m),G_평균 파고(m),W_평균 파고(m),...,평균 10cm 지중온도(°C),평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm)
0,23.2,0.0,23.4,24.0,23.0,22.0,1.2,0.0,0.7,0.0,...,21.9,21.9,21.9,22.0,21.6,21.3,19.3,16.7,1.8,3.1
1,23.1,23.2,23.5,23.5,22.9,22.9,0.5,0.5,0.3,0.3,...,22.3,22.2,22.1,22.0,21.6,21.3,19.4,16.8,1.8,3.2
2,23.0,23.6,23.2,24.3,22.8,23.2,0.5,0.3,0.3,0.2,...,23.0,22.7,22.5,22.1,21.6,21.2,19.3,16.8,3.3,4.3
3,23.1,23.7,23.3,24.3,22.8,23.3,0.4,0.3,0.2,0.2,...,23.1,22.8,22.7,22.2,21.6,21.2,19.4,16.8,2.7,5.0
4,22.8,23.5,23.1,23.7,22.6,23.4,1.0,1.1,0.7,0.7,...,23.3,23.1,23.0,22.2,21.5,21.2,19.3,16.8,5.1,7.2


In [16]:
sumRowData()

In [17]:
Row_Sum_DF[:5]

,평균 수온(°C),최고 수온(°C),최저 수온(°C),평균 유의 파고(m),평균 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec),평균기온(°C),...,평균 10cm 지중온도(°C),평균 20cm 지중온도(°C),평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm)
0,13.7,13.8,13.6,0.6,0.4,0.9,1.2,5.9,6.7,6.9,...,3.8,3.9,4.2,6.5,10.8,13.0,16.7,17.4,0.0,3.3
1,14.0,14.3,13.6,0.7,0.5,0.9,1.3,6.7,8.2,6.1,...,3.9,4.3,4.6,6.6,10.7,12.9,16.6,17.4,0.0,3.1
2,14.4,14.9,14.1,0.6,0.4,1.0,1.4,6.8,8.1,5.2,...,3.6,4.2,4.6,6.7,10.6,12.8,16.5,17.3,0.0,2.2
3,14.5,14.8,13.9,1.3,0.8,2.1,3.3,5.8,6.9,4.3,...,3.8,4.2,4.6,6.7,10.5,12.7,16.4,17.3,0.0,1.5
4,14.4,14.6,14.3,1.5,0.9,2.1,3.0,6.5,6.9,4.6,...,5.1,5.2,5.3,6.8,10.5,12.6,16.4,17.3,0.0,2.0
